In [1]:
import numpy as np
import pandas as pd
import tqdm

df = pd.read_csv("/opt/ml/input/data/train_test_data.csv")
df = df.sort_values(by=["userID", "Timestamp"]).reset_index(drop=True)

In [5]:
from datetime import date
import math
from tqdm import tqdm
import time
from datetime import datetime

def fe(df):
    #df = add_last_problem(df)
    #df = ELO_function(df)
    #df = df.drop(['left_asymptote'], axis=1)
    
    df['assessmentItemID0'] = df['assessmentItemID'].str[2]
    df['assessmentItemID1'] = df['assessmentItemID'].str[4:7]
    df['assessmentItemID2'] = df["assessmentItemID"].apply(lambda x: int(x[-2:]))
    
    df['testId0'] = df['assessmentItemID0']
    df['testId1'] = df['assessmentItemID1']
    
    df['as0_as1'] =  np.array([str(i)+'_'+str(j) for i, j in zip(list(df.assessmentItemID0), list(df.assessmentItemID1))])
    df['as0_as2'] =  np.array([str(i)+'_'+str(j) for i, j in zip(list(df.assessmentItemID0), list(df.assessmentItemID2))])
    df['as1_as2'] =  np.array([str(i)+'_'+str(j) for i, j in zip(list(df.assessmentItemID1), list(df.assessmentItemID2))])
    
    df['month'] = df.Timestamp.str[5:7]
    df['hour'] = df.Timestamp.str[11:13]

    yy = df.Timestamp.str[:4]
    mm = df.Timestamp.str[5:7]
    dd = df.Timestamp.str[8:10]

    y = [int(y) for y in list(yy)]
    m = [int(m) for m in list(mm)]
    d = [int(d) for d in list(dd)]

    df['week'] = [date(i,j,k).weekday() for i,j,k in zip(y,m,d)]
    w_h = [str(i)+'_'+j for i, j in zip(list(df.week), list(df.hour))]
    df['week_hour'] = w_h
    df['day'] = dd
    df['minute'] = df.Timestamp.str[14:16]
    df['second'] = df.Timestamp.str[17:19]

    def convert_time(s):
        timestamp = time.mktime(
            datetime.strptime(s, "%Y-%m-%d %H:%M:%S").timetuple()
        )
        return int(timestamp)

    df['Time'] = df.Timestamp.apply(convert_time)
    
    df['Timestamp'] = pd.to_datetime(df['Timestamp'])
    diff = df.loc[:, ['userID', 'Timestamp']].groupby('userID').diff().fillna(pd.Timedelta(seconds=0))
    diff = diff.fillna(pd.Timedelta(seconds=0))
    diff = diff['Timestamp'].apply(lambda x: x.total_seconds())

    df['elapsed'] = diff
    df["elapsed_log"] = df["elapsed"].apply(lambda x: math.log(x + 1))
    
    df["elapsed"] = df["elapsed"].apply(
        lambda x: x if x < 1800 and x >= 0 else 0)

    def elap(x):
        if x == 0:
            return 0
        elif x < 9:
            return 1
        elif x < 301:
            return 2
        elif x < 601:
            return 3
        elif x < 1201:
            return 4
        elif x < 1501:
            return 5
        elif x < 1801:
            return 6
        else:
            return 7

    df['elapsed_cate'] = df.elapsed.apply(lambda x : elap(x))
    
    correct_k = df.groupby(["KnowledgeTag"])["answerCode"].agg(["mean", "std"])
    correct_k.columns = ["tag_mean", "tag_std"]

    correct_a = df.groupby(["assessmentItemID0"])["answerCode"].agg(["mean", "std"])
    correct_a.columns = ["ass0_mean", "ass0_std"]
    correct_a1 = df.groupby(["assessmentItemID1"])["answerCode"].agg(["mean", "std"])
    correct_a1.columns = ["ass1_mean", "ass1_std"]
    correct_a2 = df.groupby(["assessmentItemID2"])["answerCode"].agg(["mean", "std"])
    correct_a2.columns = ["ass2_mean", "ass2_std"]

    df = pd.merge(df, correct_k, on=["KnowledgeTag"], how="left")
    df = pd.merge(df, correct_a, on=["assessmentItemID0"], how="left")
    df = pd.merge(df, correct_a1, on=["assessmentItemID1"], how="left")
    df = pd.merge(df, correct_a2, on=["assessmentItemID2"], how="left")
    
    df['test0_mean'] = df['ass0_mean']
    df['test0_std'] = df['ass0_std']
    df['test1_mean'] = df['ass1_mean']
    df['test1_std'] = df['ass1_std']

    
#     df["user_correct_answer"] = (
#         df.groupby("userID")["answerCode"]
#         .transform(lambda x: x.cumsum().shift(1))
#         .fillna(0)
#     )
#     df["user_total_answer"] = df.groupby("userID")["answerCode"].cumcount() + 1
#     df["user_acc"] = (df["user_correct_answer"] / df["user_total_answer"])
    
#     df['recAccuracy'] = -1

#     group = df.groupby("userID")

#     def set_rec_count(group):
#         group.sort_values(by='Timestamp', axis=0, inplace=True)
#         recAccuracy = group['answerCode'].rolling(10, min_periods=1).sum().shift(1)
#         group['recCount'] = recAccuracy

#         return group

#     df['recCount'] = 0

#     group = df.groupby("userID")

#     group = group.apply(set_rec_count)
#     group.reset_index(drop=True, inplace=True)

#     df['recCount'] = group['recCount']
#     df['recCount']= df['recCount'].fillna(0)
#     df['recCount'] = df['recCount'].astype(int)
    
#     def abc(x):
#         if x > 10:
#             return 10
#         else:
#             return x

#     df['abc'] = df.user_total_answer.apply(lambda x:abc(x))
#     df['recAccuracy'] = df['recCount']/df['abc']
#     df = df.drop(['abc'], axis=1)
    
#     df["solve_order"] = df.groupby(["userID", "testId"]).cumcount()
#     df["solve_order"] = (
#         df["solve_order"]
#         - df["assessmentItemID2"] * (df["solve_order"] > df["assessmentItemID2"]).apply(int)
#         + 1
#     )
    
    return df

def ELO_function(df):
    def get_new_theta(is_good_answer, beta, left_asymptote, theta, nb_previous_answers):
        return theta + learning_rate_theta(nb_previous_answers) * (
            is_good_answer - probability_of_good_answer(theta, beta, left_asymptote)
        )

    def get_new_beta(is_good_answer, beta, left_asymptote, theta, nb_previous_answers):
        return beta - learning_rate_beta(nb_previous_answers) * (
            is_good_answer - probability_of_good_answer(theta, beta, left_asymptote)
        )

    def learning_rate_theta(nb_answers):
        return max(0.3 / (1 + 0.01 * nb_answers), 0.04)

    def learning_rate_beta(nb_answers):
        return 1 / (1 + 0.05 * nb_answers)

    def probability_of_good_answer(theta, beta, left_asymptote):
        return left_asymptote + (1 - left_asymptote) * sigmoid(theta - beta)

    def sigmoid(x):
        return 1 / (1 + np.exp(-x))

    def estimate_parameters(answers_df, granularity_feature_name="assessmentItemID"):
        item_parameters = {
            granularity_feature_value: {"beta": 0, "nb_answers": 0}
            for granularity_feature_value in np.unique(
                answers_df[granularity_feature_name]
            )
        }
        student_parameters = {
            student_id: {"theta": 0, "nb_answers": 0}
            for student_id in np.unique(answers_df.userID)
        }

        print("Parameter estimation is starting...")

        for student_id, item_id, left_asymptote, answered_correctly in tqdm(
            zip(
                answers_df.userID.values,
                answers_df[granularity_feature_name].values,
                answers_df.left_asymptote.values,
                answers_df.answerCode.values,
            )
        ):
            theta = student_parameters[student_id]["theta"]
            beta = item_parameters[item_id]["beta"]

            item_parameters[item_id]["beta"] = get_new_beta(
                answered_correctly,
                beta,
                left_asymptote,
                theta,
                item_parameters[item_id]["nb_answers"],
            )
            student_parameters[student_id]["theta"] = get_new_theta(
                answered_correctly,
                beta,
                left_asymptote,
                theta,
                student_parameters[student_id]["nb_answers"],
            )

            item_parameters[item_id]["nb_answers"] += 1
            student_parameters[student_id]["nb_answers"] += 1

        print(f"Theta & beta estimations on {granularity_feature_name} are completed.")
        return student_parameters, item_parameters

    def gou_func(theta, beta):
        return 1 / (1 + np.exp(-(theta - beta)))

    df["left_asymptote"] = 0

    print(f"Dataset of shape {df.shape}")
    print(f"Columns are {list(df.columns)}")

    student_parameters, item_parameters = estimate_parameters(df)

    prob = [
        gou_func(student_parameters[student]["theta"], item_parameters[item]["beta"])
        for student, item in zip(df.userID.values, df.assessmentItemID.values)
    ]

    df["elo_prob"] = prob

    return df

def add_last_problem(df):
    new = []
    pre = df["testId"][0]
    for idx in df["testId"]:
        if pre != idx:
            new[-1] = -1
            pre = idx
        new.append(0)
    df["last_problem"] = new
    return df

df = fe(df)

In [6]:
# label encoding

cate = ['KnowledgeTag', 'month', 'hour', 'week', 'elapsed_cate', 'testId0', 'testId1',
       'assessmentItemID0', 'assessmentItemID1', 'assessmentItemID2', 'day', 'minute', 'second',
        'as0_as1', 'as0_as2', 'as1_as2', 'assessmentItemID', 'week_hour']

conti = ['elapsed' , 'tag_mean', 'tag_std', 'ass0_mean', 'ass0_std', 'elapsed_log'

'ass1_mean', 'ass1_std', 'ass2_mean', 'ass2_std', 'user_correct_answer', 'Time'

'user_total_answer', 'user_acc', 'recAccuracy', 'recCount', 'solve_order']

for i in cate:
    cate2label = {j:i for i,j in enumerate(df[i].unique())}
    df[i] = df[i].map(cate2label)

In [8]:
train = df[df['answerCode'] != -1]
test = df[df['answerCode'] == -1]

train.to_csv('../../elo.csv', index=False)
test.to_csv('../../infer.csv', index=False)

In [20]:
infer_file = df[df['rr'] == -1]
infer_file.drop(['r', 'rr'], axis=1)
infer_file.to_csv('../../../data/infer.csv', index=False)
df = df[df['rr'] != -1]
df = df.drop(['r', 'rr'], axis=1)
df.to_csv('../../../data/elo.csv', index=False)

In [62]:
# a = pd.read_csv('/opt/ml/input/data/test_2.0.csv')
# a['elo_prob'] = test['elo_prob']
a.to_csv('/opt/ml/input/data/test_elo.csv', index=False)

In [61]:
a = pd.read_csv('/opt/ml/input/data/test_elo.csv')

elo = []
for i in range(len(a)):
    if a['answerCode'].iloc[i] == -1:
        elo.append(a['elo_prob'].iloc[i-1])
    else:
        elo.append(a['elo_prob'].iloc[i])
        
a['elo_prob'] = elo

a

,userID,answerCode,KnowledgeTag,month,hour,week,elapsed,elapsed_cate,assessmentItemID0,assessmentItemID1,assessmentItemID2,user_correct_answer,user_total_answer,user_acc,recAccuracy,recCount,elo_prob
0,3,1,373,10,14,1,0.0,0,6,100,0,0.0,1,0.000000,0.000000,0,0.612587
1,3,1,373,10,14,1,26.0,2,6,100,1,1.0,2,0.500000,0.500000,1,0.559733
2,3,0,374,10,14,1,94.0,2,6,100,2,2.0,3,0.666667,0.666667,2,0.476520
3,3,0,374,10,14,1,5.0,1,6,100,3,2.0,4,0.500000,0.500000,2,0.667693
4,3,0,375,10,14,1,7.0,1,6,100,6,2.0,5,0.400000,0.400000,2,0.210991
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260109,7439,0,290,7,7,5,0.0,0,3,173,0,8.0,12,0.666667,0.700000,7,0.254957
260110,7439,1,290,7,7,5,18.0,2,3,173,1,8.0,13,0.615385,0.600000,6,0.397683
260111,7439,1,292,7,7,5,21.0,2,3,173,2,9.0,14,0.642857,0.600000,6,0.783923
260112,7439,1,292,7,7,5,89.0,2,3,173,3,10.0,15,0.666667,0.600000,6,0.780510


In [4]:
import pandas as pd
import numpy as np
train = pd.read_csv("/opt/ml/input/data/train_data.csv")
test = pd.read_csv('/opt/ml/input/data/test_data.csv')

# a = test[test['answerCode'] == -1]
# b = test[test['answerCode'] != -1]

# p = pd.read_csv('../output/p.csv')
# p_v = p['prediction']
# p_v = np.where(p_v >= 0.5, 1, 0)
# a['answerCode'] = p_v
df = pd.concat([train, test])
df = df.sort_values(by=["userID", "Timestamp"]).reset_index(drop=True)
# test = pd.concat([a, b])
# test = test.sort_values(by=["userID", "Timestamp"]).reset_index(drop=True)

In [5]:
import pandas as pd
df = pd.read_csv('../../../data/test_data.csv')

df["user_correct_answer"] = (
    df.groupby("userID")["answerCode"]
    .transform(lambda x: x.cumsum().shift(1))
    .fillna(0)
)
df["user_total_answer"] = df.groupby("userID")["answerCode"].cumcount() + 1
df["user_acc"] = (df["user_correct_answer"] / df["user_total_answer"])
df.tail(20)

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,user_correct_answer,user_total_answer,user_acc
260094,7417,A050193001,A050000193,1,2020-09-06 13:08:12,10402,0.0,12,0.000000
260095,7417,A050193002,A050000193,1,2020-09-06 13:08:23,10402,1.0,13,0.076923
260096,7417,A050193003,A050000193,0,2020-09-06 13:08:54,10402,2.0,14,0.142857
260097,7417,A050193004,A050000193,-1,2020-09-06 13:09:15,10402,2.0,15,0.133333
260098,7439,A040003001,A040000003,1,2020-01-14 08:38:16,7268,0.0,1,0.000000
260099,7439,A040003002,A040000003,1,2020-01-14 08:38:23,7268,1.0,2,0.500000
260100,7439,A040003003,A040000003,1,2020-01-14 08:39:00,7268,2.0,3,0.666667
260101,7439,A040003004,A040000003,1,2020-01-14 08:39:13,7268,3.0,4,0.750000
260102,7439,A040003005,A040000003,0,2020-01-14 08:39:48,7268,4.0,5,0.800000
260103,7439,A040197001,A040000197,1,2020-08-21 07:35:06,2132,4.0,6,0.666667


In [6]:
df[df['answerCode']==-1]

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,user_correct_answer,user_total_answer,user_acc
1035,3,A050133008,A050000133,-1,2020-10-26 13:13:57,5289,717.0,1036,0.692085
1706,4,A070146008,A070000146,-1,2020-12-27 02:47:54,9080,465.0,671,0.692996
3023,13,A070111008,A070000111,-1,2020-12-27 04:35:09,9660,915.0,1317,0.694761
4283,17,A090064006,A090000064,-1,2020-10-30 05:48:37,2611,1031.0,1260,0.818254
4670,26,A060135007,A060000135,-1,2020-10-23 11:44:18,1422,293.0,387,0.757106
...,...,...,...,...,...,...,...,...,...
260052,7395,A040122005,A040000122,-1,2020-09-08 02:05:20,10615,7.0,24,0.291667
260067,7404,A030111005,A030000111,-1,2020-10-13 09:49:18,7636,7.0,15,0.466667
260082,7416,A050193004,A050000193,-1,2020-10-04 02:44:41,10402,7.0,15,0.466667
260097,7417,A050193004,A050000193,-1,2020-09-06 13:09:15,10402,2.0,15,0.133333


In [8]:
t = pd.read_csv('../../../data/infer.csv')
t['user_correct_answer'] = df['user_correct_answer']
t['user_acc'] = df['user_acc']
t.to_csv('../../../data/infer2.csv', index=False)

In [10]:

tt.to_csv('../../../data/infer2.csv', index=False)